<a href="https://colab.research.google.com/github/naiaraAM/ML_project_UT/blob/main/Project_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up enviroment

In [2]:
!pip install -q kaggle
import pandas as pd

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/gpfs/space/software/jupyterhub/python3.9-rhel9/jupyterv2/lib/python3.9/site-packages/text_unidecode'
Check the permissions.

ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/gpfs/space/software/jupyterhub/python3.9-rhel9/jupyterv2/lib/python3.9/site-packages/pytz'
Check the permissions.



ModuleNotFoundError: No module named 'pandas'

# Download data

In [1]:
#from google.colab import files
#files.upload() # import API key

#!mkdir /root/.kaggle/
#!mv kaggle.json /root/.kaggle/kaggle.json
#!chmod 600 ~/.kaggle/kaggle.json
##!kaggle config set -n path -v"{/content}"

!kaggle competitions download -c 'playground-series-s4e6'
!unzip -q playground-series-s4e6.zip -d data
!rm playground-series-s4e6.zip

/bin/bash: line 1: kaggle: command not found
unzip:  cannot find or open playground-series-s4e6.zip, playground-series-s4e6.zip.zip or playground-series-s4e6.zip.ZIP.
rm: cannot remove 'playground-series-s4e6.zip': No such file or directory


In [ ]:
from sklearn.model_selection import train_test_split
train_data, validation_data = train_test_split(pd.read_csv('data/train.csv'), test_size=0.2, random_state=0)
test_data = pd.read_csv('data/test.csv')

print(f"The train data has {train_data.shape[0]} rows and {train_data.shape[1]} columns")
print(f"The validation data has {validation_data.shape[0]} rows and {validation_data.shape[1]} columns")
print(f"The test data has {test_data.shape[0]} rows and {test_data.shape[1]} columns")

The train data has 61214 rows and 38 columns
The validation data has 15304 rows and 38 columns
The test data has 51012 rows and 37 columns


<font color='red'>Do your code in the cell bellow your name, so we don't have merge issues. We can merge on friday's meeting</font>

## Naiara 

## Juan

## Fidan